### Definición

Siento que es necesario para nosotros saber cuanto ha aumentado la temperatura en las ciudades de "Mexico City" y "Monterrey", aunque la seria solo llega hasta el 2020 tal vez lo mejor o lógico sea predecir 5 años para que abarque a nuestra actualidad pero no creo que sea tn eficiente entoces tal vez lo mas optimo sea que pueda hacer una predicción de uno o dos años. Estos datos son importantes para mantenerno al tanto de la temperatura de nuestro país.

In [13]:
library(tidyverse)
library(fable)
library(tsibble)
library(readxl)
library(ggplot2)
library(tidyquant)
library(lubridate)

In [14]:
df <- read_csv("city_temperature.csv")
df

Rows: 2906327 Columns: 8
-- Column specification --------------------------------------------------------
Delimiter: ","
chr (4): Region, Country, State, City
dbl (4): Month, Day, Year, AvgTemperature

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


Region,Country,State,City,Month,Day,Year,AvgTemperature
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Africa,Algeria,NA,Algiers,1,1,1995,64.2
Africa,Algeria,NA,Algiers,1,2,1995,49.4
Africa,Algeria,NA,Algiers,1,3,1995,48.8
Africa,Algeria,NA,Algiers,1,4,1995,46.4
Africa,Algeria,NA,Algiers,1,5,1995,47.9
Africa,Algeria,NA,Algiers,1,6,1995,48.7
Africa,Algeria,NA,Algiers,1,7,1995,48.9
Africa,Algeria,NA,Algiers,1,8,1995,49.1
Africa,Algeria,NA,Algiers,1,9,1995,49.0


In [15]:
str(df)

spc_tbl_ [2,906,327 x 8] (S3: spec_tbl_df/tbl_df/tbl/data.frame)
 $ Region        : chr [1:2906327] "Africa" "Africa" "Africa" "Africa" ...
 $ Country       : chr [1:2906327] "Algeria" "Algeria" "Algeria" "Algeria" ...
 $ State         : chr [1:2906327] NA NA NA NA ...
 $ City          : chr [1:2906327] "Algiers" "Algiers" "Algiers" "Algiers" ...
 $ Month         : num [1:2906327] 1 1 1 1 1 1 1 1 1 1 ...
 $ Day           : num [1:2906327] 1 2 3 4 5 6 7 8 9 10 ...
 $ Year          : num [1:2906327] 1995 1995 1995 1995 1995 ...
 $ AvgTemperature: num [1:2906327] 64.2 49.4 48.8 46.4 47.9 48.7 48.9 49.1 49 51.9 ...
 - attr(*, "spec")=
  .. cols(
  ..   Region = col_character(),
  ..   Country = col_character(),
  ..   State = col_character(),
  ..   City = col_character(),
  ..   Month = col_double(),
  ..   Day = col_double(),
  ..   Year = col_double(),
  ..   AvgTemperature = col_double()
  .. )
 - attr(*, "problems")=<externalptr> 


In [16]:
state_counts <- table(df$State)
state_counts


Additional Territories                Alabama                 Alaska 
                  6787                  37060                  27795 
               Arizona               Arkansas             California 
                 26484                  18530                  46325 
              Colorado            Connecticut               Delaware 
                 37060                  18530                   5751 
  District of Columbia                Florida                Georgia 
                 18530                  61341                  37060 
                Hawaii                  Idaho               Illinois 
                  9265                  18530                  37060 
               Indiana                   Iowa                 Kansas 
                 37060                  18530                  27795 
              Kentucky              Louisiana                  Maine 
                 27795                  33593                  18530 
              Maryl

In [17]:
# Dropeamos la variable State
df <- subset(df, select = -State)

In [18]:
year_counts <- table(df$Year)
year_counts


   200    201   1995   1996   1997   1998   1999   2000   2001   2002   2003 
    89    351 118616 118951 118656 119082 119355 119682 119355 119355 119140 
  2004   2005   2006   2007   2008   2009   2010   2011   2012   2013   2014 
118645 117895 117647 115428 114908 114247 115627 113740 111993 111021 109018 
  2015   2016   2017   2018   2019   2020 
107498 107237 106946 106698 106337  38810 

In [19]:
# Filtramo para usar las filas con datos con años menos de 1995
df <- df %>%
  filter(Year >= 1995)

In [20]:
# Convertimos la temperatura registrada en Farenheit a Celsius
df$AvgTemperature <- (df$AvgTemperature - 32) / 1.8
df

Region,Country,City,Month,Day,Year,AvgTemperature
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Africa,Algeria,Algiers,1,1,1995,17.888889
Africa,Algeria,Algiers,1,2,1995,9.666667
Africa,Algeria,Algiers,1,3,1995,9.333333
Africa,Algeria,Algiers,1,4,1995,8.000000
Africa,Algeria,Algiers,1,5,1995,8.833333
Africa,Algeria,Algiers,1,6,1995,9.277778
Africa,Algeria,Algiers,1,7,1995,9.388889
Africa,Algeria,Algiers,1,8,1995,9.500000
Africa,Algeria,Algiers,1,9,1995,9.444444


In [21]:
day_counts <- table(df$Day)
day_counts


    0     1     2     3     4     5     6     7     8     9    10    11    12 
    8 95649 95654 95656 95655 95653 95653 95653 95653 95653 95652 95647 95644 
   13    14    15    16    17    18    19    20    21    22    23    24    25 
95616 95355 95356 95352 95352 95351 95352 95349 95349 95349 95344 95344 95343 
   26    27    28    29    30    31 
95345 95343 95344 89367 87335 55511 

In [22]:
# Convirtiendo cada renglon de día, mes y año en uno solo con los 3
df$date <- ymd(paste(df$Year, df$Month, df$Day, sep = "-"))
df

Warning message:
" 8 failed to parse."


Region,Country,City,Month,Day,Year,AvgTemperature,date
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<date>
Africa,Algeria,Algiers,1,1,1995,17.888889,1995-01-01
Africa,Algeria,Algiers,1,2,1995,9.666667,1995-01-02
Africa,Algeria,Algiers,1,3,1995,9.333333,1995-01-03
Africa,Algeria,Algiers,1,4,1995,8.000000,1995-01-04
Africa,Algeria,Algiers,1,5,1995,8.833333,1995-01-05
Africa,Algeria,Algiers,1,6,1995,9.277778,1995-01-06
Africa,Algeria,Algiers,1,7,1995,9.388889,1995-01-07
Africa,Algeria,Algiers,1,8,1995,9.500000,1995-01-08
Africa,Algeria,Algiers,1,9,1995,9.444444,1995-01-09


In [23]:
# dropeamos las columnas de fechas que ya no nos sirven
df <- df[, !names(df) %in% c("Year", "Day", "Month")]

In [24]:
# Country seleccionado
selected_continent <- c('Mexico')


# Filtrar filas basadas en continentes y años seleccionados
rows_df <- df %>%
  filter(Country %in% selected_continent)

rows_df

Region,Country,City,AvgTemperature,date
<chr>,<chr>,<chr>,<dbl>,<date>
North America,Mexico,Guadalajara,-72.77778,1995-01-01
North America,Mexico,Guadalajara,-72.77778,1995-01-02
North America,Mexico,Guadalajara,-72.77778,1995-01-03
North America,Mexico,Guadalajara,-72.77778,1995-01-04
North America,Mexico,Guadalajara,-72.77778,1995-01-05
North America,Mexico,Guadalajara,14.55556,1995-01-06
North America,Mexico,Guadalajara,-72.77778,1995-01-07
North America,Mexico,Guadalajara,-72.77778,1995-01-08
North America,Mexico,Guadalajara,15.83333,1995-01-09


In [25]:
rows_city_count <- table(rows_df$City)
rows_city_count


Guadalajara Mexico City   Monterrey 
       8540        9265        9265 

In [26]:
# Eliminar todas las filas que contienen la ciudad de Guadalajara
rows_df <- rows_df %>%
  filter(City != "Guadalajara")

In [27]:
rows_df

Region,Country,City,AvgTemperature,date
<chr>,<chr>,<chr>,<dbl>,<date>
North America,Mexico,Mexico City,14.27778,1995-01-01
North America,Mexico,Mexico City,13.38889,1995-01-02
North America,Mexico,Mexico City,14.22222,1995-01-03
North America,Mexico,Mexico City,14.05556,1995-01-04
North America,Mexico,Mexico City,14.88889,1995-01-05
North America,Mexico,Mexico City,16.77778,1995-01-06
North America,Mexico,Mexico City,15.00000,1995-01-07
North America,Mexico,Mexico City,14.44444,1995-01-08
North America,Mexico,Mexico City,14.50000,1995-01-09


In [ ]:
rows_df = rows_df %>% select(Region, Country, City, AvgTemperature, date) %>% as_tsibble(index=date, regular = TRUE, key = City)
rows_df